In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
application_one = application_df.copy()

In [3]:
application_two = application_df.copy()

In [4]:
application_three = application_df.copy()

In [5]:
#Add more hidden layers

# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_one = application_df.drop(columns=['EIN', 'NAME'])

In [6]:
# Determine the number of unique values in each column.
unique_counts = application_one.nunique()
unique_counts

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [7]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_type_counts = application_one['APPLICATION_TYPE'].value_counts()
application_type_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [8]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_cutoff = 500
application_types_to_replace = application_type_counts[application_type_counts < application_cutoff].index

# Replace in dataframe
for app in application_types_to_replace:
    application_one['APPLICATION_TYPE'] = application_one['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_one['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_one = pd.get_dummies(application_one)

In [10]:
# Split our preprocessed data into our features and target arrays
X = application_one.drop('IS_SUCCESSFUL', axis=1).values
y = application_one['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation='relu', input_dim=X_train.shape[1]))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=25, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

C:\Users\LabUser\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 80)                  │           8,720 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 30)                  │           2,430 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 25)                  │             775 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              26 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,951 (46.68 KB)

 Trainable params: 11,951 (46.68 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6911 - loss: 0.6038 - val_accuracy: 0.7407 - val_loss: 0.5474
Epoch 2/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7269 - loss: 0.5523 - val_accuracy: 0.7389 - val_loss: 0.5479
Epoch 3/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7293 - loss: 0.5490 - val_accuracy: 0.7413 - val_loss: 0.5505
Epoch 4/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7292 - loss: 0.5500 - val_accuracy: 0.7352 - val_loss: 0.5495
Epoch 5/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7293 - loss: 0.5476 - val_accuracy: 0.7423 - val_loss: 0.5452
Epoch 6/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7402 - loss: 0.5393 - val_accuracy: 0.7392 - val_loss: 0.5455
Epoch 7/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7342 - loss: 0.5418 - val_accuracy: 0.7409 - val_loss: 0.5477
Epoch 8/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7341 - loss: 0.5415 - val_accu

686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7407 - loss: 0.5330 - val_accuracy: 0.7402 - val_loss: 0.5562
Epoch 52/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7393 - loss: 0.5339 - val_accuracy: 0.7423 - val_loss: 0.5531
Epoch 53/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7439 - loss: 0.5289 - val_accuracy: 0.7413 - val_loss: 0.5560
Epoch 54/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7396 - loss: 0.5312 - val_accuracy: 0.7405 - val_loss: 0.5505
Epoch 55/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7376 - loss: 0.5334 - val_accuracy: 0.7416 - val_loss: 0.5528
Epoch 56/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7390 - loss: 0.5328 - val_accuracy: 0.7403 - val_loss: 0.5600
Epoch 57/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7387 - loss: 0.5350 - val_accuracy: 0.7407 - val_loss: 0.5533
Epoch 58/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7414 - loss: 0.5309 - val_accuracy:

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 1ms/step - accuracy: 0.7299 - loss: 0.5690
Loss: 0.5690122842788696, Accuracy: 0.7298833727836609


In [16]:
#dropping only EIN column

application_two = application_df.copy()

# Drop the non-beneficial ID columns, 'EIN'
application_two = application_df.drop(columns=['EIN'])

In [17]:
# Determine the number of unique values in each column.
unique_counts = application_two.nunique()
unique_counts

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [18]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_type_counts = application_two['APPLICATION_TYPE'].value_counts()
application_type_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [19]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_cutoff = 500
application_types_to_replace = application_type_counts[application_type_counts < application_cutoff].index

# Replace in dataframe
for app in application_types_to_replace:
    application_two['APPLICATION_TYPE'] = application_two['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_two['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [20]:
# Convert categorical data to numeric with `pd.get_dummies`
application_two = pd.get_dummies(application_two)

In [21]:
# Split our preprocessed data into our features and target arrays
X = application_two.drop('IS_SUCCESSFUL', axis=1).values
y = application_two['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [23]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation='relu', input_dim=X_train.shape[1]))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

C:\Users\LabUser\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                      │ (None, 80)                  │       1,574,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 30)                  │           2,430 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 1)                   │              31 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,576,621 (6.01 MB)

 Trainable params: 1,576,621 (6.01 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [25]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 12s 14ms/step - accuracy: 0.7035 - loss: 0.5739 - val_accuracy: 0.7937 - val_loss: 0.4483
Epoch 2/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.9468 - loss: 0.1453 - val_accuracy: 0.6132 - val_loss: 0.7206
Epoch 3/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.9612 - loss: 0.0978 - val_accuracy: 0.6316 - val_loss: 0.6569
Epoch 4/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.9643 - loss: 0.0852 - val_accuracy: 0.7289 - val_loss: 0.5111
Epoch 5/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.9622 - loss: 0.0882 - val_accuracy: 0.7556 - val_loss: 0.4929
Epoch 6/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.9619 - loss: 0.0864 - val_accuracy: 0.7624 - val_loss: 0.4879
Epoch 7/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.9636 - loss: 0.0865 - val_accuracy: 0.6423 - val_loss: 0.6127
Epoch 8/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.9663 - loss: 0.0828 -

686/686 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.9692 - loss: 0.0744 - val_accuracy: 0.8007 - val_loss: 3.3398
Epoch 51/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - accuracy: 0.9704 - loss: 0.0712 - val_accuracy: 0.8001 - val_loss: 2.7923
Epoch 52/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - accuracy: 0.9694 - loss: 0.0716 - val_accuracy: 0.8007 - val_loss: 3.3315
Epoch 53/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - accuracy: 0.9704 - loss: 0.0737 - val_accuracy: 0.7870 - val_loss: 2.1843
Epoch 54/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - accuracy: 0.9698 - loss: 0.0742 - val_accuracy: 0.7959 - val_loss: 2.5294
Epoch 55/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.9676 - loss: 0.0765 - val_accuracy: 0.7897 - val_loss: 2.5379
Epoch 56/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.9701 - loss: 0.0719 - val_accuracy: 0.7979 - val_loss: 2.8988
Epoch 57/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.9688 - loss: 0.0746 - 

686/686 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.9690 - loss: 0.0725 - val_accuracy: 0.8016 - val_loss: 9.7509
Epoch 100/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - accuracy: 0.9691 - loss: 0.0712 - val_accuracy: 0.8003 - val_loss: 7.9497


In [26]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 2ms/step - accuracy: 0.7206 - loss: 0.8402
Loss: 0.8401550650596619, Accuracy: 0.7205539345741272


In [40]:
#add more nuerons to a hidden layer

application_three = application_df.copy()

In [41]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_three = application_df.drop(columns=['EIN', 'NAME'])

In [42]:
# Determine the number of unique values in each column.
unique_counts = application_three.nunique()
unique_counts

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [43]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_type_counts = application_three['APPLICATION_TYPE'].value_counts()
application_type_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [44]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_cutoff = 500
application_types_to_replace = application_type_counts[application_type_counts < application_cutoff].index

# Replace in dataframe
for app in application_types_to_replace:
    application_three['APPLICATION_TYPE'] = application_three['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_three['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [45]:
# Convert categorical data to numeric with `pd.get_dummies`
application_three = pd.get_dummies(application_three)

In [46]:
# Split our preprocessed data into our features and target arrays
X = application_three.drop('IS_SUCCESSFUL', axis=1).values
y = application_three['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [47]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [48]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation='relu', input_dim=X_train.shape[1]))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=50, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

C:\Users\LabUser\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_7 (Dense)                      │ (None, 80)                  │           8,720 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 50)                  │           4,050 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 1)                   │              51 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,821 (50.08 KB)

 Trainable params: 12,821 (50.08 KB)

 Non-trainable params: 0 (0.00 B)

In [49]:
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [50]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6874 - loss: 0.6112 - val_accuracy: 0.7378 - val_loss: 0.5550
Epoch 2/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7253 - loss: 0.5582 - val_accuracy: 0.7383 - val_loss: 0.5499
Epoch 3/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7254 - loss: 0.5557 - val_accuracy: 0.7383 - val_loss: 0.5501
Epoch 4/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7245 - loss: 0.5506 - val_accuracy: 0.7418 - val_loss: 0.5462
Epoch 5/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7342 - loss: 0.5456 - val_accuracy: 0.7422 - val_loss: 0.5487
Epoch 6/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7311 - loss: 0.5447 - val_accuracy: 0.7407 - val_loss: 0.5504
Epoch 7/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7298 - loss: 0.5450 - val_accuracy: 0.7431 - val_loss: 0.5483
Epoch 8/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7320 - loss: 0.5474 - val_accu

686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7376 - loss: 0.5337 - val_accuracy: 0.7413 - val_loss: 0.5524
Epoch 52/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7388 - loss: 0.5357 - val_accuracy: 0.7413 - val_loss: 0.5523
Epoch 53/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7384 - loss: 0.5289 - val_accuracy: 0.7420 - val_loss: 0.5513
Epoch 54/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7390 - loss: 0.5321 - val_accuracy: 0.7416 - val_loss: 0.5657
Epoch 55/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7392 - loss: 0.5361 - val_accuracy: 0.7407 - val_loss: 0.5603
Epoch 56/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7424 - loss: 0.5269 - val_accuracy: 0.7407 - val_loss: 0.5558
Epoch 57/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7409 - loss: 0.5278 - val_accuracy: 0.7403 - val_loss: 0.5556
Epoch 58/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7418 - loss: 0.5310 - val_accuracy:

In [51]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 968us/step - accuracy: 0.7289 - loss: 0.5696
Loss: 0.5696125626564026, Accuracy: 0.728863000869751
